#### Name: Qiwen Ma

Date: 12/03/2018

Version: 3.0

Environment: Python 3.6.0 and Jupyter notebook

Libraries used:
* pandas (for dataframe, included in Anaconda Python 2.7) 
* re (for regular expression, included in Anaconda Python 2.7) 

## 1.  Import libraries 

In [ ]:
# To import libraries needed in this assessment
import pandas as pd
import re

## 2. Parse PDF File

The first step is to load the pdf file using pdftables.

In [ ]:
from pdftables import get_tables
pdfFile = './health.pdf'
pdfobj = open(pdfFile,'rb')
tables = get_tables(pdfobj)

Then to skip the rows in summary and to seprate the data by pages.

In [ ]:
for table in tables:
    for row in table[5:]:
        if 'SUMMARY' in row[0]:
            break
        print (row)
    print ('==========================\n')

Using the regular expression to settle the data.

In [ ]:
reg = re.compile("^\s*$")
reg1 = re.compile("[a-zA-Z]|\(|\)|\s")
reg2 = re.compile("^[0-9]$|^100$|^[1-9][0-9]x?$|^–$|^$|^[0-9]x$")
reg3 = re.compile("[0-9]|–")
reg4 = re.compile("(100)|([1-9][0-9])")
reg5 = re.compile("^.*[0-9]+.*")

First, to find out all the country names and spreate them from numbers.To save each country name in each new list. 

In [ ]:
for table in tables:
    for row in table[5:]:
        if 'SUMMARY' in row[0]:
            break
        if reg.match(row[0]) != None:
            break 
        newlist = []
        newlist1 = []
        # using regular expression to find out all the data that is characters only
        country = "".join(re.findall(reg1,row[0]))
        newlist.append(country) 
        print(newlist)

Second, to separate the numbers,which are following the country names in the last page.

In [ ]:
for table in tables:
    for row in table[5:]:
        if 'SUMMARY' in row[0]:
            break
        if reg.match(row[0]) != None:
            break 
        newlist = []
        newlist1 = []
        country = "".join(re.findall(reg1,row[0]))
        newlist.append(country)       
        
        str1 = ''
        #use search the matched data using regular expression and keep as a string in the first column
        if reg5.search(row[0]) != None: 
            for num in row[0]:
                if reg2.match(num) != None:
                    str1 += num
            #to split the string using regular expression
            a = re.split(reg4,str1)
            #to append the number spilt above to each list that contains country names
            for i in range(0,len(a)):
                if a[i] != None and a[i] !='':
                    newlist.append(a[i])
            print(newlist)

Third, to separate the number that are aggregated togther in different places and append them into the list above.

In [ ]:
for table in tables:
    for row in table[5:]:
        if 'SUMMARY' in row[0]:
            break
        if reg.match(row[0]) != None:
            break 
        newlist = []
        newlist1 = []
        country = "".join(re.findall(reg1,row[0]))
        newlist.append(country)       
        
        str1 = ''
        if reg5.search(row[0]) != None: 
            for num in row[0]:
                if reg2.match(num) != None:
                    str1 += num                   
            a = re.split(reg4,str1)
            for i in range(0,len(a)):
                if a[i] != None and a[i] !='':
                    newlist.append(a[i])
        
        for info in row[1:]: 
            str = ''
            #to match the data that are with two number,100,number/numbers with 'x',desh and null 
            if re.match(reg2,info) != None:
                newlist.append(info)
            else:
                for number in info:
                    #to match the number one by one, if the number is a single value combine it with the next one 
                    str += number
                    if re.match(reg2,str) == None:
                        newlist.append(str[:-1])
                        str = number
                if str != None:
                    newlist.append(str)
        print(newlist)

Then, save the data in a dataframe.

In [ ]:
data={}
for table in tables:
    for row in table[5:]:
        if 'SUMMARY' in row[0]:
            break
        if reg.match(row[0]) != None:
            break 
        newlist = []
        newlist1 = []
        country = "".join(re.findall(reg1,row[0]))
        newlist.append(country)       
        
        str1 = ''
        if reg5.search(row[0]) != None: 
            for num in row[0]:
                if reg2.match(num) != None:
                    str1 += num                   
            a = re.split(reg4,str1)
            for i in range(0,len(a)):
                if a[i] != None and a[i] !='':
                    newlist.append(a[i])
        
        for info in row[1:]: 
            str = ''
            if re.match(reg2,info) != None:
                newlist.append(info)
            else:
                for number in info:
                    str += number
                    if re.match(reg2,str) == None:
                        newlist.append(str[:-1])
                        str = number
                if str != None:
                    newlist.append(str)
            data[newlist[0]]=newlist[1:]
data = pd.DataFrame(data)
#exchange the  row index and column names in the original dataframe
df = data.T
df

For making the data to be accurate, here is a function to compare and check the exact two columns of data. For this assignment, it can be observed from the pdf file that the data in 'total', 'urban' and 'rural' column have their data characteristic. For most of the numbers in 'total' column is smaller than the numbers in 'urban' due to the data in 'rural'. The presentage data of 'rural' will affect the 'total' presentage. It need to have a check of the 'urban' and 'total' columns when the 'urban' data is larger than 'total'.

In [ ]:
def check(name_list,list1,list2):
    for i in range(0,len(list1)):
        if list1[i] != '–' and list2[i] != '–' and int(list1[i]) > int(list2[i]):
            print("To check", " ", name_list[i], " ", list1[i] , " " , list2[i])   

In [ ]:
total = df.iloc[0:202,3].tolist()
urban = df.iloc[0:202,4].tolist()
name = df.index.tolist()
check(name,total,urban)

After observation and check the data to the original pdf file, there is only one value in the dataframe is still not correct. Then replace the value from '71' to '7' and '8' to '18'.

In [ ]:
df.loc['Ethiopia'][3] = '7'
df.loc['Ethiopia'][4] = '18'
df.loc['Ethiopia']

Finally, to delete the 'x' values that are with numbers and replace the desh values to null in the dataframe.

In [ ]:
for col_idx in range(len(df.columns)):
    for row_idx in range(len(df)):
        if 'x' in df.iloc[row_idx,col_idx]:
            df.iloc[row_idx,col_idx] = df.iloc[row_idx,col_idx].strip('x')
df

In [ ]:
df_result = df.replace('–',' ')
df_result

In [ ]:
df_result.to_csv('health.csv') 

## 3. Summary

After finishing this assignment task, what I have learned is concluded as follow:
1. It is different to use search, findall and match function with the regular expression. In this assignment task, I use all these methods in different situation. It depends on the data that I need to extract.
2. It can change the data type into string for further use. In this assignment I assigned the data to be a new string using the regular expression and then separate the number data.